In [ ]:
%run ../supportvectors-common.ipynb

We walk through a vanilla AutoEncoder on the MNIST dataset

First the imports

In [2]:

import torch
from torch.utils.data import DataLoader
from sklearn.mixture import GaussianMixture
from svlearn.auto_encoders.vanilla_auto_encoder_mnist import (AutoencoderMnist,
                                                        train_autoencoder,
                                                        visualize_reconstruction,
                                                        get_latent_representations
)
from svlearn.auto_encoders.auto_encoder_util import (   sample_from_gmm,
                                                        generate_images_from_latent,
                                                        visualize_generated_images,
                                                        visualize_interpolations)
import torchvision.datasets as datasets

Load the path for mnist data and results

In [3]:
mnist_data_path = '/home/chandar/data'
model_path = '/home/chandar/results'

In [4]:
mnist_trainset = datasets.MNIST(root=mnist_data_path, train=True, download=True, transform=None)

train_dataset = mnist_trainset.data[:-10000].reshape(-1, 1, 28, 28) / 255.
eval_dataset = mnist_trainset.data[-10000:].reshape(-1, 1, 28, 28) / 255.

In [5]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(eval_dataset, batch_size=128, shuffle=False)

Identify device (either cuda or cpu)

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoencoderMnist().to(device)

Run training on the vanilla autoencoder

In [ ]:
# Train autoencoder
train_autoencoder(model, train_loader, val_loader, num_epochs=10, learning_rate=1e-3, device=device)

Visualize the reconstruction (top row being original images and bottom being reconstructed)

In [ ]:
# Create random shuffles of the val loader to visualize different samples each time.
val_loader = DataLoader(eval_dataset, batch_size=128, shuffle=True) 
# Call this to monitor reconstruction
visualize_reconstruction(model, val_loader, device=device)

Generate images from hidden vectors using the decoder of the autoencoder

In [ ]:

# Collect latent representations from training data
latent_data = get_latent_representations(model, train_loader, device=device)

# Fit a Gaussian Mixture Model with 10 components
gmm = GaussianMixture(n_components=10)
gmm.fit(latent_data)

# Sample from the GMM
latent_samples = sample_from_gmm(gmm, num_samples=10)

# Generate images from the latent samples
generated_images = generate_images_from_latent(model, latent_samples, device=device)

# Visualize the generated images
visualize_generated_images(generated_images)


Visualize interpolations between the reconstruction of two images

In [ ]:

# Create random shuffle of train dataset to pick random 2 images every time.
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
first_train_batch = next(iter(train_loader))
# Visualize interpolations
# Assuming image1 and image2 are samples from your dataset (PIL images already transformed to tensor)
image1 = first_train_batch[0]  # First image
image2 = first_train_batch[1]  # Second image

# Visualize interpolation between the two images
visualize_interpolations(model, image1, image2, num_steps=10, device=device)